In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,66,2024-03-13,1,111.923077,96.552308,0.537092,10.835385,24.633846,0.175631,115.338462,...,12.712308,25.932308,0.224046,114.061538,0,3,2,3,3,2024
1,66,2024-03-13,-4,114.246154,98.616923,0.542385,11.327692,24.870769,0.182292,115.001538,...,13.051563,24.968750,0.197641,112.323437,0,3,2,2,28,2024
2,66,2024-03-13,3,115.015385,96.169231,0.561677,10.741538,25.696923,0.175138,119.546154,...,11.225000,21.982812,0.218359,114.968750,0,3,2,2,8,2024
3,66,2024-03-13,-2,111.538462,95.449231,0.531031,10.560000,24.378462,0.187415,115.063077,...,11.164615,23.427692,0.180031,121.326154,0,3,2,2,4,2024
4,66,2024-03-13,-2,113.723077,96.752308,0.556923,11.536923,23.798462,0.184385,116.689231,...,11.448438,25.464063,0.208891,118.929688,0,3,2,2,6,2024
5,66,2024-03-13,-1,107.061538,97.047692,0.526308,11.923077,21.581538,0.166600,110.046154,...,12.823077,22.340000,0.187185,108.023077,0,3,2,2,5,2024
6,65,2024-03-13,1,118.546875,99.228125,0.554609,12.018750,27.140625,0.183297,118.365625,...,10.524615,22.020000,0.198523,119.433846,0,3,2,2,10,2024
7,67,2024-03-13,2,117.287879,100.078788,0.564833,11.803030,19.560606,0.211091,116.239394,...,11.250794,23.920635,0.172159,118.142857,0,3,2,3,14,2024
8,65,2024-03-13,-1,119.625000,99.926563,0.538047,10.856250,27.704687,0.212797,118.859375,...,12.543750,26.906250,0.192703,110.454687,0,3,2,3,1,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,0
1,2024-03-13,TOR,DET,2.300000,1.645161,0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1
3,2024-03-13,CHI,IND,2.400000,1.602410,0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0
5,2024-03-13,CHO,MEM,1.833333,2.000000,0
6,2024-03-13,GSW,DAL,3.250000,1.359712,0
7,2024-03-13,LAL,SAC,1.740741,2.140000,0
8,2024-03-13,ATL,POR,1.617284,2.360000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,0
1,2024-03-13,TOR,DET,2.300000,1.645161,0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1
3,2024-03-13,CHI,IND,2.400000,1.602410,0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0
5,2024-03-13,CHO,MEM,1.833333,2.000000,1
6,2024-03-13,GSW,DAL,3.250000,1.359712,0
7,2024-03-13,LAL,SAC,1.740741,2.140000,0
8,2024-03-13,ATL,POR,1.617284,2.360000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,0
1,2024-03-13,TOR,DET,2.300000,1.645161,0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1
3,2024-03-13,CHI,IND,2.400000,1.602410,0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0
5,2024-03-13,CHO,MEM,1.833333,2.000000,1
6,2024-03-13,GSW,DAL,3.250000,1.359712,0
7,2024-03-13,LAL,SAC,1.740741,2.140000,0
8,2024-03-13,ATL,POR,1.617284,2.360000,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,1
1,2024-03-13,TOR,DET,2.300000,1.645161,0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1
3,2024-03-13,CHI,IND,2.400000,1.602410,0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0
5,2024-03-13,CHO,MEM,1.833333,2.000000,1
6,2024-03-13,GSW,DAL,3.250000,1.359712,0
7,2024-03-13,LAL,SAC,1.740741,2.140000,0
8,2024-03-13,ATL,POR,1.617284,2.360000,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,1.0
1,2024-03-13,TOR,DET,2.300000,1.645161,1.0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1.0
3,2024-03-13,CHI,IND,2.400000,1.602410,0.0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0.0
5,2024-03-13,CHO,MEM,1.833333,2.000000,1.0
6,2024-03-13,GSW,DAL,3.250000,1.359712,0.0
7,2024-03-13,LAL,SAC,1.740741,2.140000,1.0
8,2024-03-13,ATL,POR,1.617284,2.360000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-13,BRK,ORL,3.250000,1.359712,1.0
1,2024-03-13,TOR,DET,2.300000,1.645161,1.0
2,2024-03-13,DEN,MIA,1.617284,2.360000,1.0
3,2024-03-13,CHI,IND,2.400000,1.602410,0.0
4,2024-03-13,CLE,NOP,3.200000,1.370370,0.0
5,2024-03-13,CHO,MEM,1.833333,2.000000,1.0
6,2024-03-13,GSW,DAL,3.250000,1.359712,0.0
7,2024-03-13,LAL,SAC,1.740741,2.140000,1.0
8,2024-03-13,ATL,POR,1.617284,2.360000,1.0
